# D213 - Advanced Data Analytics
### NLM3 Task 1: Time Series Modeling
#### Advanced Data Analytics — D213
#### PRFA — NLM3
> André Davis
> StudentID: 010630641
> MSDA
>
> Competencies
> 4030.7.2 : Time Series Analysis
> The graduate applies time series models in generating forecasts. 

#### Table of Contents
<ul>
    <li><a href="#research-question">A1: Research Question</a></li>
    <li><a href="#objectives">A2: Objectives and Goals of Analysis</a></li>
    <li><a href="#assumptions">B: Assumptions of Time Series Model</a></li>
    <li><a href="#visualizations">C1: Time Series Visualization</a></li>
    <li><a href="#time-step-description">C2: Description of Time Step Formatting</a></li>
    <li><a href="#stationary-of-series">C3: Stationarity of Series</a></li>
    <li><a href="#data-preparation">C4: Data Preparation & Explanation</a></li>
    <li><a href="#copy-of-prepared-data">C5: Copy of Prepared Data Set</a></li>
    <li><a href="#annotated-findings-and-visualizations">D1: Annotated Findings & Visualizations</a></li>
    <li><a href="#arima-model-of-time-series-data">D2: ARIMA Model of Time Series Data</a></li>
    <li><a href="#forcast">D3: Forecast</a></li>
    <li><a href="#analysis">D4: Analysis Output & Calculations</a></li>
    <li><a href="#arima-code">D5: ARIMA Model Code</a></li>
    <li><a href="#results-of-analysis">E1: Results of Analysis</a></li> 
    <li><a href="#forcast-visualizations">E2: Visualization of Forecast</a></li> 
    <li><a href="#recommendations">E3: Recommended Action</a></li>
    <li><a href="#reporting">F: Reporting</a></li>
    <li><a href="#code-references">G: Code References</a></li>
    <li><a href="#source-references">H: Source References</a></li>    
</ul>

<a id="research-question"></a>
# A1: Research Question

The research question explored in this report is: "Is it possible to accurately and effectively forecast the daily revenues of [`WGU`](https://www.wgu.edu/) Hospital System in a manner that aligns closely with the actual observed daily revenues?"

<a id="objectives"></a>
# A2: Objectives and Goals of Analysis

The objective of this analysis is to accurately predict daily revenues for the [`WGU`](https://www.wgu.edu) Hospital System utilizing the available [`dataset`](https://access.wgu.edu/ASP3/aap/content/fj30d8sm59fc83ed9xsi.zip). The analysis employs an 80-20 split of the dataset, using the initial 80% as a training set to forecast the remaining 20%, which serves as a test set. This approach allows for the use of observed values as a benchmark for comparison. An ARIMA time series model will be utilized to analyze the training set and project the values in the test set.

> ARIMA is a "model runs *d* rounds of differencing to make the time series more stationary, then it applies a regular ARMA Model. When making forecasts, it uses this ARMA model, then it adds back the terms that were subtracted by differencing."(Géron, 2022).

<a id="assumptions"></a>
# B: Assumptions of Time Series Model

>The submission accurately summarizes each of the assumptions of a time series model. The summary includes stationarity and autocorrelated data.

Assumptions of the ARMA Family of Models, including ARIMA include:

 1. Data must not include outliers and/or anomalies 
 2. The time series data should exhibit stationary, devoid of any trends or seasonal fluctuations.
 3. Datapoints in the past must be indicative of future datapoints in behavior.
 4. The data must reflect a single variable and be classified as uni-variate to be modelled.
 5. The data must be auto-correlated for ARIMA model to perform forecasting via Autoregressive Component 

Some observational notes about the [`WGU Medical Time Series Dataset`](https://access.wgu.edu/ASP3/aap/content/fj30d8sm59fc83ed9xsi.zip) and the assumption point #3. Because this is a Performance Assessment for graduate school, the data presented for this activity is pretty limited. There are 730 rows and because this is a daily record, this equals 2 years of daily data. It may or may not be enough information to know if the past data indicates future behavior.

<a id="visualizations"></a>
# C1: Time Series Visualization

In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

medical_daily_revenue = pd.read_csv('./medical_time_series.csv')
medical_daily_revenue.info()

print(f'\nTotal Records: [{len(medical_daily_revenue)}]')

#### Visualization of the Medical Daily Revenue Dataset

In [108]:
def chart_time_series(data: pd.DataFrame, title: str) -> None:
    plt.figure(figsize=(16,5))
    plt.xlabel('Day')
    plt.ylabel('Daily Revenue - USD(M)')
    plt.title(f'{title} first 2 years of Daily Revenue(M)')
    
    x_days = data['Day']
    y_revenue = data['Revenue']
    plt.plot(x_days, y_revenue)
    
    #generate trend line
    polynomial_coefficients = np.polyfit(x_days, y_revenue, 1)
    polynomial_function = np.poly1d(polynomial_coefficients) 
    
    plt.plot(x_days, polynomial_function(x_days), c='r', linestyle='--', label='Trendline')
    plt.legend()
    
    plt.show()
    
chart_time_series(medical_daily_revenue, 'WGU Revenue')

<a id="time-step-description"></a>
# C2: Description of Time Step Formatting

The format of the time series data is pretty straight forward. We have two variables(columns) called `Day` and `Revenue`. The data represents revenue in the millions per day for two years since the opening of the *WGU Hospital*. The data does not contain any missing revenue values or days as it's a complete data set starting at Day 1 and continuing through Day 731, which is two years worth of data. The fact the total data is 731 and not 730 could indicate that the 2 years spanned through a [leap year](https://en.wikipedia.org/wiki/Leap_year) in February.

<a id="stationary-of-series"></a>
# C3: Stationary of Series

To test whether or not the data is stationary or not we will back-up the visual check with the Augmented Dickery-Fuller (ADF) test.

> "Augmented Dickey Fuller test (ADF Test) is a common statistical test used to test whether a given Time series is stationary or not. It is one of the most commonly used statistical test when it comes to analyzing the stationary of a series." (Prabhakaran, 2022)

In [109]:
'''
Augmented Dickey-Fuller Documentation: 
https://www.statsmodels.org/dev/generated/statsmodels.tsa.stattools.adfuller.html

statsmodels.tsa.stattools.adfuller(
    x, 
    maxlag=None, 
    regression='c', 
    autolag='AIC', 
    store=False, 
    regresults=False
)
'''

def execute_adfuller(data: pd.Series, title: str) -> None:
    adf, p_value, used_lag, nobs, critical_values, icbest = adfuller(data)
    
    print(f'ADF Statistic: {adf}')
    print(f'p-value: {p_value}')
    print(f'Number of lags used: {used_lag}')
    print(f'Number of observations: {nobs}')
    print(f'Critical Values: {critical_values}')
    print(f'Maximized Information Criterion: {icbest}')
    
    print('\n')
    print('p-value Stationary Results:')
    print('=============================================================')
    is_data_stationary = p_value < .05
    
    print(f'p-value check: {round(p_value, 4)} < .05')
    if is_data_stationary:
        print(f'{title} Time Series data is likely stationary.')
    else:
        print(f'{title} Time Series data is likely non-stationary.')
    print('=============================================================')
    
    print('\n')
    print('adf (test statistic) Results:')
    print('=============================================================')
    
    print(f"adf check (1%): {round(adf, 4)} < {round(critical_values['1%'], 4)}")
    print(f"adf check (5%): {round(adf, 4)} < {round(critical_values['5%'], 4)}")
    print(f"adf check (10%): {round(adf, 4)} < {round(critical_values['10%'], 4)}")
    
    if adf < critical_values['1%']:
        print(f'{title} Time Series data is stationary at 1% significance level.')
    elif adf < critical_values['5%']:    
        print(f'{title} Time Series data is stationary at 5% significance level.')
    elif adf < critical_values['10%']:
        print(f'{title} Time Series data is stationary at 10% significance level.')
    else:
        print(f'{title} Time Series data is likely non-stationary.')

execute_adfuller(medical_daily_revenue['Revenue'], 'WGU Revenue')


<a id="data-preparation"></a>
# C4: Data Preparation & Explanation

Time Series Analysis will require a little bit of data prepping. After preparing the data will also be split into training and testing data, so we can validate how well the model for the Time Series is performing.

Steps:

 1. Transform `Revenue` data into a stationary time series format by getting 'First discrete difference of object' using the [`.diff()`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.diff.html) function of [`Pandas`](https://pandas.pydata.org/).
 2. Removing missing values after transformation 
 3. Split transformed data into training and test datasets with an 80%/20% split. 

In [110]:
'''
Use diff() function to get the Discrete Differences and then perform .dropna() to remove missing values.
'''

discrete_differences = medical_daily_revenue.diff()
discrete_differences = discrete_differences.dropna()

#put correct days back instead of diff for day count
#discrete_differences['Day'] = medical_daily_revenue['Day']

diff_revenue = discrete_differences['Revenue']

execute_adfuller(diff_revenue, 'Transformed WGU Revenue')

In [111]:
'''
Visual Check that Revenue Data is now Stationary
'''

plt.figure(figsize=(16,5))
plt.xlabel('Day')
plt.ylabel('Revenue')
plt.title('Stationary Visualization Check')

plt.plot(discrete_differences)

This chart does not show any signs of trending or seasonality. The visual check confirms the p-value and adf value check of stationary.

In [112]:
'''
Train and Test data splitting 80/20.
'''

train, test = train_test_split(discrete_differences, test_size=.2, shuffle=False, random_state=999)

print(train.head())
print(test.head())

<a id="copy-of-prepared-data"></a>
# C5: Copy of Prepared Data Set

In [113]:
'''Saving Training Data'''
train.to_csv('./time-series-training-data.csv')

'''Saving Test Data'''
test.to_csv('./time-series-testing-data.csv')


<a id="annotated-findings-and-visualizations"></a>
# D1: Annotated Findings & Visualizations

<a id="arima-model-of-time-series-data"></a>
# D2: ARIMA Model of Time Series Data

<a id="forcast"></a>
# D3: Forecast

<a id="analysis"></a>
# D4: Analysis Output & Calculations

<a id="arima-code"></a>
# D5: ARIMA Model Code

<a id="results-of-analysis"></a>
# E1: Results of Analysis

<a id="forcast-visualizations"></a>
# E2: Visualization of Forecast

<a id="recommendations"></a>
# E3: Recommended Action

<a id="reporting"></a>
# F: Reporting

<a id="code-references"></a>
# G: Code References

<a id="source-references"></a>
# H: Source References

 * Géron, A. (2022). Hands-On Machine Learning with Scikit-Learn, Keras, and Tensorflow: Concepts, Tools, and Techniques to Build Intelligent Systems. <br /><br />
 * Prabhakaran, S. (2022). Augmented Dickey Fuller Test (ADF Test) – must read guide. Machine Learning Plus. https://www.machinelearningplus.com/time-series/augmented-dickey-fuller-test/ 